In [ ]:
import sys
import os

if 'root_dir' not in globals():
    # rootディレクトリへのパスを設定
    root_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
    os.chdir(root_dir)



In [ ]:
query = '韓国料理'

# 記事を取得する

In [ ]:
# 自作したコードをimportする
from my_codes import fetch_articles


In [ ]:
query_keys_all_df, selected_all_user_data = fetch_articles.main(
    query = query, size=100, batches=10000,interval=1
)

# 前処理をする

In [ ]:
from my_codes import preprocessing

In [ ]:
preprocessing.main(query=query)

# データベースに保存する

In [ ]:
import my_codes.database_operation as db_op

In [ ]:
db_op.add_csv_to_database(query=query)

# クエリを実行

In [ ]:
from my_codes.database_setting import Engine
from my_codes.database_setting import Base

from my_codes.notes_database import Notes

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=Engine)
session = Session()

In [ ]:
import pandas as pd

In [ ]:
# ORMを使って特定のカラムを取得
database_keys = session.query(Notes.key).all()
session.close()
# リストをDataFrameに変換
df_database = pd.DataFrame(database_keys, columns=['key'])